In [1]:
# This cell allows then notebook to discover the package TunnelingExperiment
# Only run this cell once. If run a second time, hit "Kernel->Reset" to restart
# the notebook, then run once.

import os
moduledir = os.path.dirname( os.getcwd() )

os.chdir(moduledir) # Move up one directory so we can import modules

#Import libraries
%matplotlib inline
from ipywidgets import interactive,fixed
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np


# Fitting

Example of pulling data to make an interactive plot of an image

In [2]:
data_path = 'BLG-CHTC/raw-data'

# Change to VT and VB
VT = np.load( os.path.join(data_path,'VT.npy') )
VB = np.load( os.path.join(data_path,'VB.npy') )

# Change to d1 and Wtip
heights = np.load( os.path.join(data_path,'height.npy') )
Wtips = np.load( os.path.join(data_path,'Wtip.npy') )

# Import Current
Current = np.load( os.path.join(data_path, 'Current.npy') )

In [3]:
## Temporary cell to show how to plot the background as well.
def image_plot(im,kx,kxrange,ky,kyrange,alpha):
    fig = plt.figure(2,frameon=False)
    xindex = (np.abs(kx - kxrange)).argmin()
    yindex = (np.abs(ky - kyrange)).argmin()
    
    data = plt.imshow(image[10][10],cmap=cm.gray,origin='lower',
                     aspect='auto',alpha=1-alpha,extent=(domain[0],domain[-1],domain[0],domain[-1]))
    
    fit = plt.imshow(image[xindex][yindex],cmap=cm.RdYlGn,origin='lower',
                aspect='auto',alpha=alpha,extent=(domain[0],domain[-1],domain[0],domain[-1]))
    plt.show()

In [3]:
def plot_dIdV(I,VT,VB,d1,d1_range,Wtip,Wtip_range,alpha,norm=False):
    """
    Plots a color map of dI/dV spectrum (VT vs VB).

    Parameters
    ----------
    show:       Boolean; Select "True" to show the image. Select "False" (default)
                to simply generate it. Useful if you would just like to save the image.

    save:       Boolean; Select "True" to save to (file location)

    norm:       Boolean; Select "True" to normalize and plot (dI/dV)/(I/VT) spectrum.
    """
    d1_index = (np.abs(d1 - d1_range)).argmin()
    Wtip_index = (np.abs(Wtip - Wtip_range)).argmin()  
    
    dIdV = np.gradient(I[d1_index,Wtip_index,:,:],axis=0) # dI/dV

    if norm == True:
        IV = I / VT[:,np.newaxis] # I/V
        dIdV = dIdV / IV

    fig, ax = plt.subplots(figsize=(7,6),frameon=False)

    dIdV_plot = plt.imshow(dIdV,cmap=cm.RdYlGn,origin='lower',
                            aspect='auto',alpha=alpha,extent=(VB[0],VB[-1],1e3*VT[0],1e3*VT[-1]))
    fig.suptitle('$dI/dV$\n Tip Height ={} nm Work Function = {} meV'.format(d1_range[d1_index],Wtip_range[Wtip_index]))
    cbar = fig.colorbar(dIdV_plot,label='$dI/dV$ (S)')
    ax.set_xlabel('Gate Voltage (V)')
    ax.set_ylabel('Tip Voltage (mV)')

    plt.show()

In [4]:
interactive_plot = interactive(plot_dIdV, I=fixed(Current), VT=fixed(VT), VB=fixed(VB),
                               d1=(heights[0], heights[-1]),d1_range=fixed(heights),
                               Wtip=(Wtips[0],Wtips[-1]),Wtip_range=fixed(Wtips),
                              alpha=(0.,1.))
output = interactive_plot.children[-1]
output.layout.height = '300px'
interactive_plot

interactive(children=(FloatSlider(value=14.0, description='d1', max=26.0, min=2.0), FloatSlider(value=4900.0, …

* Tip workfunction > Sample workfunction
* Tip attracts electrons, acts as negative gate. Opens a gap.
* To close, offset the neImagine applying positive back gate voltage. To close gap, either apply a negative voltage. OR 